## Importing PySpark

In [ ]:
import pyspark 
sc = pyspark.SparkContext('local[*]')

## More Examples

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

df = spark.read.json('tip.json')

rdd = df.rdd.map(tuple)

rdd = rdd.map(lambda r: r[3])

In [ ]:
rdd.count()

In [ ]:
rdd.take(5)

How many “big” (10+ letters), “medium” (5 – 9 letters), “small” and “tiny” (1 letter) words are used?

In [ ]:
from operator import add

def find_word_size(line):
    map_d = {'tiny': 0, 'small': 0, 'medium': 0, 'big': 0}
    for word in line.split():
        if len(word) >= 10:
            map_d['big'] += 1
        elif len(word) >= 5:
            map_d['medium'] += 1
        elif len(word) > 1:
            map_d['small'] += 1
        else:
            map_d['tiny'] += 1
    return map_d

mapResult = rdd.map(find_word_size)

In [ ]:
mapResult.take(5)

In [ ]:
def reduce_word_size(map_d, map_r = {'tiny': 0, 'small': 0, 'medium': 0, 'big': 0}):
    for k in map_d.keys():
        map_r[k] += map_d[k]
    return map_r

mapResult.reduce(reduce_word_size)

## Using Combiners

In [ ]:
data = sc.parallelize( [('A', 200), ('B', 400), ('C', 500), ('D', 1000), ('E', 5000), ('F', 1000), ('B', 4000), ('D', 3000), ('F', 50000), ('G', 1000)] )

sumCount = data.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [ ]:
sumCount.collect()

---------

## Build an Inverted Index

In [ ]:
rdd = df.rdd.map(tuple)
rdd = rdd.map(lambda r: (r[3], r[0]))

rdd.take(3)

In [ ]:
def find_word_business(tuple_b):
    return [(w, tuple_b[1]) for w in tuple_b[0].split()]

mapResult = rdd.flatMap(find_word_business)
mapResult.take(10)

In [ ]:
filterResult = mapResult.filter(lambda x: len(x[0]) >= 10)
filterResult.take(10)

In [ ]:
filterResult.reduceByKey(lambda x, y: x + ', ' + y).collect()

--------------------

## Partition Function

Example from: https://medium.com/parrot-prediction/partitioning-in-apache-spark-8134ad840b0

In [ ]:
transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'James', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Germany'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
    {'name': 'Phil', 'amount': 50, 'country': 'Argentina'},
]

def country_partitioner(country):
    return hash(country)

rdd = sc.parallelize(transactions) \
        .map(lambda el: (el['country'], el)) \
        .partitionBy(2, country_partitioner)

In [ ]:
print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitions structure: {}".format(rdd.glom().collect()))